In [1]:
import sys
sys.path.append('../vocaldetection/')
import sklearn
import utils
import json
import os
import pandas as pd
import numpy as np

In [2]:
# Read features and labels
FEAT_PATH = os.environ["FEAT_PATH"]
PIECES = os.environ["PIECES_JSON"]

In [3]:
music_files = []

with open(PIECES) as json_file:  
    data = json.load(json_file)

    for music in data.keys():
        music_files.append(FEAT_PATH+music)

In [4]:
train_features = []
train_labels = []


for tf in music_files:
    # Load VGGish audio embeddings
    #vggish = pd.read_csv(tf+"_VGGish_PCA.csv",index_col=None, header=None)
    #vggish = vggish.values
    
    # Load MFCCs features
    mfccs = pd.read_csv(tf+"_mfcc.csv",index_col=None, header=None)
    mfccs = mfccs.values.T
    
    # Load VocalVariance features
    vv = pd.read_csv(tf+"_vocalvar.csv",index_col=None, header=None)
    vv = vv.values.T

    # Read labels for each frame
    labels = pd.read_csv(tf+"_lbl.csv",index_col=None, header=None)
    labels = labels.values[0]
    
    feature_vector = []
    for idx in range(len(labels)):
        feature_vector.append(np.concatenate((mfccs[idx], vv[idx]), axis=0))
        
        
    # Store the feature vector and corresponding label in integer format
    for idx in range(len(feature_vector)):
        train_features.append(feature_vector[idx])
        train_labels.append(labels[idx])


In [17]:
X = np.array(train_features)
y = np.array(train_labels)

ValueError: could not convert string to float: '[ 1.07115547e+02 -3.47272873e+01  1.57313881e+01 -7.95158529e+00\n  1.71852474e+01 -1.61360092e+01  1.72172463e+00  5.96364737e+00\n -1.08871949e+00 -7.30616331e-01 -6.06386423e+00  1.73046386e+00\n -1.50515115e+00  1.03846169e+01  9.42243099e+00  8.85262012e+00\n  8.56095219e+00  7.02073193e+00  5.01579237e+00  5.14964819e+00\n  3.47757244e+00  5.02254057e+00  3.22590995e+00  3.13083696e+00\n  3.60949039e+00  3.08055925e+00  1.30667923e+02  1.69300842e+01\n  3.14402905e+01  1.25162954e+01  2.96958160e+01 -7.04921961e-01\n  1.21851339e+01  1.27911100e+01  9.26175213e+00  4.96795130e+00\n  5.92182815e-01  1.05797224e+01  7.20575094e+00  1.07875473e+02\n -3.68285027e+01  1.72950649e+01 -9.97012806e+00  1.90751572e+01\n -1.62606220e+01  2.87016296e+00  5.67637873e+00 -1.13273811e+00\n -4.24645036e-01 -6.02352333e+00  1.47536767e+00 -1.74072909e+00\n  3.49201083e-01 -5.08942604e-01  4.30020332e-01  1.43476486e-01\n  3.43437791e-01 -6.21106811

In [6]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [7]:
# Split dataset into train, validation and test sets

In [8]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.model_selection import KFold # import KFold
seed = 7
kf = KFold(n_splits=10, random_state=seed) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

10

In [11]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 2704  2705  2706 ... 27036 27037 27038] TEST: [   0    1    2 ... 2701 2702 2703]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [2704 2705 2706 ... 5405 5406 5407]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [5408 5409 5410 ... 8109 8110 8111]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [ 8112  8113  8114 ... 10813 10814 10815]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [10816 10817 10818 ... 13517 13518 13519]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [13520 13521 13522 ... 16221 16222 16223]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [16224 16225 16226 ... 18925 18926 18927]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [18928 18929 18930 ... 21629 21630 21631]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [21632 21633 21634 ... 24333 24334 24335]
TRAIN: [    0     1     2 ... 24333 24334 24335] TEST: [24336 24337 24338 ... 27036 27037 27038]


In [12]:
# Fit models

# Evaluate models

In [15]:
X[0][0]

'[ 1.07115547e+02 -3.47272873e+01  1.57313881e+01 -7.95158529e+00\n  1.71852474e+01 -1.61360092e+01  1.72172463e+00  5.96364737e+00\n -1.08871949e+00 -7.30616331e-01 -6.06386423e+00  1.73046386e+00\n -1.50515115e+00  1.03846169e+01  9.42243099e+00  8.85262012e+00\n  8.56095219e+00  7.02073193e+00  5.01579237e+00  5.14964819e+00\n  3.47757244e+00  5.02254057e+00  3.22590995e+00  3.13083696e+00\n  3.60949039e+00  3.08055925e+00  1.30667923e+02  1.69300842e+01\n  3.14402905e+01  1.25162954e+01  2.96958160e+01 -7.04921961e-01\n  1.21851339e+01  1.27911100e+01  9.26175213e+00  4.96795130e+00\n  5.92182815e-01  1.05797224e+01  7.20575094e+00  1.07875473e+02\n -3.68285027e+01  1.72950649e+01 -9.97012806e+00  1.90751572e+01\n -1.62606220e+01  2.87016296e+00  5.67637873e+00 -1.13273811e+00\n -4.24645036e-01 -6.02352333e+00  1.47536767e+00 -1.74072909e+00\n  3.49201083e-01 -5.08942604e-01  4.30020332e-01  1.43476486e-01\n  3.43437791e-01 -6.21106811e-02  1.20982766e-01 -3.85089107e-02\n -7.71256

In [13]:
# Create a scale object
scaler = sklearn.preprocessing.StandardScaler()

# Learn the parameters from the training data only
scaler.fit(train_features)

# Apply the learned parameters to the training and test sets:
train_features_scaled = scaler.transform(train_features)

ValueError: could not convert string to float: '[ 1.07115547e+02 -3.47272873e+01  1.57313881e+01 -7.95158529e+00\n  1.71852474e+01 -1.61360092e+01  1.72172463e+00  5.96364737e+00\n -1.08871949e+00 -7.30616331e-01 -6.06386423e+00  1.73046386e+00\n -1.50515115e+00  1.03846169e+01  9.42243099e+00  8.85262012e+00\n  8.56095219e+00  7.02073193e+00  5.01579237e+00  5.14964819e+00\n  3.47757244e+00  5.02254057e+00  3.22590995e+00  3.13083696e+00\n  3.60949039e+00  3.08055925e+00  1.30667923e+02  1.69300842e+01\n  3.14402905e+01  1.25162954e+01  2.96958160e+01 -7.04921961e-01\n  1.21851339e+01  1.27911100e+01  9.26175213e+00  4.96795130e+00\n  5.92182815e-01  1.05797224e+01  7.20575094e+00  1.07875473e+02\n -3.68285027e+01  1.72950649e+01 -9.97012806e+00  1.90751572e+01\n -1.62606220e+01  2.87016296e+00  5.67637873e+00 -1.13273811e+00\n -4.24645036e-01 -6.02352333e+00  1.47536767e+00 -1.74072909e+00\n  3.49201083e-01 -5.08942604e-01  4.30020332e-01  1.43476486e-01\n  3.43437791e-01 -6.21106811

In [ ]:
test_features_scaled = scaler.transform(test_features)

In [ ]:
# save the scaler to disk
filename = 'scaler_features.sav'
joblib.dump(scaler, filename)

In [ ]:
# Use scikit-learn to train a model with the training features we've extracted

# Lets use a SVC with folowing C parameters: 
params = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

for c in params:
    clf = sklearn.svm.SVC(C=c)

    # Fit (=train) the model
    clf.fit(train_features_scaled, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_SVM_'+str(c)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf.predict(test_features_scaled)
    
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf.predict(test_features_scaled)

In [ ]:
# Biased Guess Accuracy
ones = np.ones(len(predictions))
accuracy = sklearn.metrics.accuracy_score(test_labels, ones)
print(accuracy)

In [ ]:
# SVM Accuracy
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)

In [ ]:
# lets compute the show the confusion matrix:
cm = sklearn.metrics.confusion_matrix(test_labels, predictions)
print(cm)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(cm, interpolation='nearest', cmap='gray')
for i, line in enumerate(cm):
    for j, l in enumerate(line):
        ax.text(j, i, l, size=20, color='green')
ax.set_xticks(range(len(cm)))
ax.set_xticklabels(labels)
ax.set_yticks(range(len(cm)))
ax.set_yticklabels(labels)
ax.set_ylabel('True label')
ax.set_xlabel('Predicted label')
plt.show()

### Random Forest Classifier without scaler

In [ ]:
params = [100, 200, 500, 1000]

for estimators in params:
    clf3 = RandomForestClassifier(n_estimators=estimators)

    # Fit (=train) the model
    clf3.fit(train_features, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_RF_'+str(estimators)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf3, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf3.predict(test_features)
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf3.predict(test_features)

In [ ]:
# We can use sklearn to compute the accuracy score
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)